# しんりゅうパターン出力

In [1]:
import os
import random
import pandas as pd
import numpy as np
from scipy.optimize import NonlinearConstraint, Bounds, minimize

COUNT_OF_SAMPLES = 100

CHARACTER_COUNT = 5
CHARACTER_HP_MAX = 9999
CHARACTER_HP_MIN = 1999
WAVE_DAMAGE_MAX = 7999
WAVE_DAMAGE_MIN = 5999
THUNDER_DAMAGE_MAX = 5999
THUNDER_DAMAGE_MIN = 3999
HEAL_MAX = 3999
HEAL_MIN = 1499

### 珊瑚の指輪　装備時の制限関数

In [2]:
def finalHP_coral(vars):
    """
    vars = (wave, thunder, HP, heal) すべて実数とみなす。
    珊瑚の指輪のみ装備時の 最終HP を返す。
    """
    w, t, H, r = vars

    # 1ターン目ダメージ
    wave_damage = w / 3.0
    if wave_damage >= H:
        # タイダルウエイブで死
        return 0.0
    
    # 生存中なら wave 後のHP
    HP_after_wave = H - wave_damage

    # 回復 (死んでなければ)
    # min( HP_after_wave + r, H )
    HP_after_heal = HP_after_wave + r
    if HP_after_heal > H:
        HP_after_heal = H

    # いなづま (ダメージ2倍)
    thunder_damage = 2.0 * t
    HP_final = HP_after_heal - thunder_damage

    return HP_final

### ダイアの鎧　装備時の制限関数

In [5]:
def finalHP_diamond(vars):
    w, t, H, r = vars
    # 1ターン目(タイダルウエイブ)
    if w >= H:
        return 0.0
    HP_after_wave = H - w

    # 回復
    HP_after_heal = HP_after_wave + r
    if HP_after_heal > H:
        HP_after_heal = H

    # 2ターン目(いなづま) ダイアの鎧で 1/3
    thunder_damage = t / 3.0
    return HP_after_heal - thunder_damage

### 珊瑚の指輪・ダイアの鎧　装備時の制限関数

In [6]:
def finalHP_both(vars):
    w, t, H, r = vars
    # 1ターン目(タイダルウエイブ: 1/3)
    wave_damage = w / 3.0
    if wave_damage >= H:
        return 0.0
    HP_after_wave = H - wave_damage

    # 回復
    HP_after_heal = HP_after_wave + r
    if HP_after_heal > H:
        HP_after_heal = H

    # 2ターン目(いなづま: 珊瑚2倍 × ダイア1/3 = 2/3 倍)
    thunder_damage = (2.0 * t) / 3.0
    return HP_after_heal - thunder_damage

### scipyを使用した数理最適化
#### やっていること
- hp_max、heal を最小化した時の wave, thunder 値 を求める
- wave, thunder を最大化した時の hp_max、heal 値 を求める
- 装備を変えて上記を求める、など。作業用で作成したものでコードを直接弄りながら使用していました。再利用性は全く考えていません

In [7]:
%%time

# --- 制約: finalHP >= 1 ---
nlc = NonlinearConstraint(finalHP_coral, 1, np.inf)
# nlc = NonlinearConstraint(finalHP_diamond, 1, np.inf)
# nlc = NonlinearConstraint(finalHP_both, 1, np.inf)

# --- バウンド: wave, thunder, HP, heal ---
# wave in [5999, 7999]
# thunder in [3999, 5999]
# HP in [2999, 9999]
# heal in [1999, 3999]

bounds = Bounds(
    lb=[5999.0, 3999.0, 1999.0, 1499.0],
    ub=[7999.0, 5999.0, 9999.0, 3999.0])

# --- 追加の線形制約 wave < 3*HP ---
# すなわち w - 3H < 0
# => w - 3H <= -ε としたいが Boundsは "w >= 6000" とセットなので
#    ここでは scipy.optimize.LinearConstraint を使う
from scipy.optimize import LinearConstraint

# yasubeiいったんコメントアウトする
# さんごの指輪、もしくは両方装備のみの場合は以下のようにする
A = np.array([[1.0, 0.0, -3.0, 0.0]])  # w - 3H
lc = LinearConstraint(A, -np.inf, -1e-9)  
# -1e-9 は「実数計算なのでギリギリ負にする」という工夫

# ダイアの鎧のみの場合は以下のようにする
# A = np.array([[1.0, 0.0, -1.0, 0.0]])  # w - H
# lc = LinearConstraint(A, -np.inf, -1e-9)  
# いなづまの値は回復があるので線形制約をつけられない

# 目的関数（とりあえず可行解を探すだけ -> 定数0でOK）
def obj(vars):
    return 0.0

# 目的関数(生存可能な範囲内でなるべく wave を小さくする)
def obj_min_wave(vars):
    w, t, H, r = vars
    return w

# 目的関数(生存可能な範囲内でなるべく HPと回復量 を小さくする)
def obj_min_hp(vars):
    w, t, H, r = vars
    return H

# 目的関数(生存可能な範囲内でなるべく HPと回復量 を小さくする)
def obj_min_heal(vars):
    w, t, H, r = vars
    return r

# 目的関数(生存可能な範囲内でなるべく HPと回復量 を小さくする)
def obj_min_hp_heal(vars):
    w, t, H, r = vars
    return H + r

# 目的関数(生存可能な範囲内でなるべく wave を大きくする)
def obj_neg_wave(vars):
    w, t, H, r = vars
    return -w

# 目的関数(生存可能な範囲内でなるべく thunder を大きくする)
def obj_neg_thunder(vars):
    w, t, H, r = vars
    return -t

# 目的関数(生存可能な範囲内でなるべく wave と thunder を大きくする)
def obj_neg_wave_plus_thunder(vars):
    w, t, H, r = vars
    return -(w + t)
    
# 初期解（ざっくり）
x0 = np.array([5999.0, 3999.0, 3000.0, 2999.0]) # 珊瑚のゆびわのみ
# x0 = np.array([7999.0, 3999.0, 4325.0, 1499.0]) # 両方装備
# x0 = np.array([7999.0, 3999.0, 8000.0, 1499.0]) # ダイアの鎧のみ

res = minimize(
    fun=obj_min_heal,
    x0=x0,
    method='SLSQP',
    bounds=bounds,
    constraints=[nlc, lc],
    options={'maxiter':10000, 'ftol':1e-9, 'disp':True}
)

if res.success:
    w_opt, t_opt, H_opt, r_opt = res.x
    #print("===== 珊瑚の指輪のみ: 一例の可行解を探索 =====")
    #print("===== ダイアの鎧のみ: 一例の可行解を探索 =====")
    print("===== 両方装備: 一例の可行解を探索 =====")
    print(f"wave   = {w_opt}")
    print(f"thunder= {t_opt}")
    print(f"HP     = {H_opt}")
    print(f"heal   = {r_opt}")
    print(f"finalHP= {finalHP_coral(res.x)}") # 珊瑚のゆびわのみ
    # print(f"finalHP_diamond= {finalHP_diamond(res.x)}") # ダイアの鎧のみ
    # print(f"finalHP_both= {finalHP_both(res.x)}") # 珊瑚のゆびわとダイアの鎧を装備
else:
    print("No feasible solution found.")


Optimization terminated successfully    (Exit mode 0)
            Current function value: 1499.0000000000464
            Iterations: 22
            Function evaluations: 121
            Gradient evaluations: 21
===== 両方装備: 一例の可行解を探索 =====
wave   = 5999.0
thunder= 3999.0
HP     = 8499.66666666662
heal   = 1499.0000000000464
finalHP= 1.0
CPU times: user 14.5 ms, sys: 0 ns, total: 14.5 ms
Wall time: 14.4 ms
